# PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import clear_output

In [2]:
# load csvs to dataframe
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
len_test = len(df_test)

# we need both for the temporal features
df = pd.concat([df_train, df_test], ignore_index=True)

In [3]:
df.head()

,AssortmentType,CloudCover,Date,Events,HasPromotions,IsHoliday,IsOpen,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,...,NumberOfCustomers,NumberOfSales,Precipitationmm,Region,Region_AreaKM2,Region_GDP,Region_PopulationK,StoreID,StoreType,WindDirDegrees
0,General,8.0,01/03/2016,Rain-Snow,0,0,1,1,NaN,100,...,495.0,5676.0,0.0,7,9643,17130,2770,1000,Hyper Market,23
1,General,8.0,02/03/2016,Snow,0,0,1,0,NaN,87,...,608.0,8111.0,0.0,7,9643,17130,2770,1000,Hyper Market,56
2,General,8.0,04/03/2016,Rain,0,0,1,0,NaN,81,...,665.0,8300.0,0.0,7,9643,17130,2770,1000,Hyper Market,22
3,General,6.0,05/03/2016,NaN,0,0,1,-3,NaN,80,...,630.0,7154.0,0.0,7,9643,17130,2770,1000,Hyper Market,108
4,General,6.0,06/03/2016,NaN,0,0,0,0,NaN,93,...,0.0,0.0,0.0,7,9643,17130,2770,1000,Hyper Market,46


In [4]:
# show sample row
df.iloc[0]

AssortmentType                     General
CloudCover                               8
Date                            01/03/2016
Events                           Rain-Snow
HasPromotions                            0
IsHoliday                                0
IsOpen                                   1
Max_Dew_PointC                           1
Max_Gust_SpeedKm_h                     NaN
Max_Humidity                           100
Max_Sea_Level_PressurehPa             1032
Max_TemperatureC                         2
Max_VisibilityKm                        19
Max_Wind_SpeedKm_h                      21
Mean_Dew_PointC                         -1
Mean_Humidity                           82
Mean_Sea_Level_PressurehPa            1030
Mean_TemperatureC                        1
Mean_VisibilityKm                       11
Mean_Wind_SpeedKm_h                     16
Min_Dew_PointC                          -2
Min_Humidity                            70
Min_Sea_Level_PressurehPa             1029
Min_Tempera

In [5]:
## Missing Values
# cloud coverage: 0 if no events, 8 if events
for row in range(len(df)):
    if row % 10000 == 0:
        clear_output()
        print("Working on row {}".format(row))
    if np.isnan(df.loc[row, 'CloudCover']):
        if df.loc[row, 'Events'] is np.nan:
            df.loc[row, 'CloudCover'] = 0
        else:
            df.loc[row, 'CloudCover'] = 8

# max gust speed = max wind speed
df.Max_Gust_SpeedKm_h = df.Max_Gust_SpeedKm_h.fillna(df.Max_Wind_SpeedKm_h)

Working on row 560000


In [6]:
## Date Features
# convert date to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# add features
# df['DayN']=df['Date'].dt.dayofyear    # non credo possa servire
df['DayOfWeek']=df['Date'].dt.dayofweek
df['Month']=df['Date'].dt.month
df['Week']=df['Date'].dt.weekofyear
df['Quarter']=df['Date'].dt.quarter


In [7]:
## Temporal Features
# create columns
df['IsOpen_yesterday'] = np.empty(len(df))
df['IsOpen_tomorrow'] = np.empty(len(df))
df['IsHoliday_yesterday'] = np.empty(len(df))
df['IsHoliday_tomorrow'] = np.empty(len(df))
df['NumberOfSales_yesterday'] = np.empty(len(df))
df['NumberOfSales_lastweek'] = np.empty(len(df))
df['NumberOfSales_lastmonth'] = np.empty(len(df))

for store in df.StoreID.unique():
    clear_output()
    print("Working on {}".format(store))
    temp = df.loc[df.StoreID == store]
    # switch index to timestamps to make this easier
    oldindex = temp.index
    temp.index = temp['Date']
    
    temp['IsOpen_yesterday'] = temp.IsOpen.rolling(window='1d',closed='left', min_periods=1).sum()
    temp['IsOpen_tomorrow'] = temp.IsOpen.rolling(window='1d',closed='left', min_periods=1).sum().shift(-2, '1d')
    temp['IsHoliday_yesterday'] = temp.IsHoliday.rolling(window='1d',closed='left', min_periods=1).sum()
    temp['IsHoliday_tomorrow'] = temp.IsHoliday.rolling(window='1d',closed='left', min_periods=1).sum().shift(-2, '1d')
    temp['NumberOfSales_yesterday'] = temp.NumberOfSales.rolling(window='1d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastweek'] = temp.NumberOfSales.rolling(window='7d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastmonth'] = temp.NumberOfSales.rolling(window='30d',closed='left', min_periods=1).sum()
    
    # put it back in the dataframe
    temp.index = oldindex
    df.loc[df.StoreID == store] = temp
    
# Attenzione: i valori di tomorrow nel test sono sputtanati a NaN
# vanno messi a mano qui o cambiato il modo di calcolo
    
# drop rows at the beginning where we have no past information
# NB: possiamo fare a meno se togliamo quelle feature
df = df.iloc[30:]

Working on 1748


In [8]:
# One-Hot Encoding
# nb: pd.get_dummies rimuove le colonne direttamente

# ## StoreId
# df = pd.get_dummies(df, columns=['StoreID'], prefix='StoreID')

 ## StoreType
# df = pd.get_dummies(df, columns=['StoreType'], prefix='StoreType')

 ## AssortmentType
# df = pd.get_dummies(df, columns=['AssortmentType'], prefix='AssortmentType')

# ## Region
# df = pd.get_dummies(df, columns=['Region'], prefix='Region')

# ## Events
# df = pd.get_dummies(df, columns=['Events'], prefix='Events', dummy_na=True)
### inutile se possiamo usare categorie con decision tree

# numeric features to categories (strings)
# df.StoreID = df.StoreID.astype(str)
# df.Region = df.Region.astype(str)


In [9]:
# Drop useless columns
#df = df.drop('Date', axis=1)
df = df.drop('NumberOfCustomers', axis=1)  
df = df.drop('WindDirDegrees', axis=1)

# questi non cambiano mai, teniamo regione e population
df = df.drop('Region_AreaKM2', axis=1)
df = df.drop('Region_GDP', axis=1)
#df = df.drop('Region_PopulationK', axis=1)

In [10]:
# Drop rows with IsOpen = 0
# -> number of sales is always = 0 
df = df[df.IsOpen == 1]
df = df.drop('IsOpen', axis=1)

In [11]:
df.head()

,AssortmentType,CloudCover,Date,Events,HasPromotions,IsHoliday,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,...,Month,Week,Quarter,IsOpen_yesterday,IsOpen_tomorrow,IsHoliday_yesterday,IsHoliday_tomorrow,NumberOfSales_yesterday,NumberOfSales_lastweek,NumberOfSales_lastmonth
31,General,6.0,2016-04-02,NaN,0,0,-2,21.0,87,1009,...,4,13,2,0.0,0.0,1.0,0.0,0.0,44106.0,186345.0
34,General,1.0,2016-04-05,NaN,0,0,-5,45.0,65,1013,...,4,14,2,0.0,1.0,1.0,0.0,0.0,40680.0,182020.0
35,General,6.0,2016-04-06,Rain,0,0,-4,29.0,70,1015,...,4,14,2,1.0,1.0,0.0,0.0,6981.0,39072.0,189001.0
36,General,7.0,2016-04-07,NaN,0,0,-2,32.0,70,1014,...,4,14,2,1.0,1.0,0.0,0.0,5222.0,35239.0,184113.0
37,General,7.0,2016-04-08,NaN,0,0,-2,24.0,75,1013,...,4,14,2,1.0,1.0,0.0,0.0,6509.0,29841.0,181603.0


In [12]:
print(list(df.columns))

['AssortmentType', 'CloudCover', 'Date', 'Events', 'HasPromotions', 'IsHoliday', 'Max_Dew_PointC', 'Max_Gust_SpeedKm_h', 'Max_Humidity', 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm', 'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm', 'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity', 'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM', 'NearestCompetitor', 'NumberOfSales', 'Precipitationmm', 'Region', 'Region_PopulationK', 'StoreID', 'StoreType', 'DayOfWeek', 'Month', 'Week', 'Quarter', 'IsOpen_yesterday', 'IsOpen_tomorrow', 'IsHoliday_yesterday', 'IsHoliday_tomorrow', 'NumberOfSales_yesterday', 'NumberOfSales_lastweek', 'NumberOfSales_lastmonth']


In [13]:
# save preprocessed data
df[:-len_test].to_csv('preprocessed_train.csv',index=False)
df[-len_test:].to_csv('preprocessed_test.csv', index=False)

In [14]:
### REMEMBER
# le sales di testing vanno aggiunte e sistemate a runtime